In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import cross_val_score




In [34]:
hit = pd.read_csv("Hitters.csv")
df =hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[["League","Division","NewLeague"]])
y = df["Salary"]
x_ = df.drop(["Salary","League","Division","NewLeague"], axis = 1).astype("float64")
x = pd.concat([x_,dms[["League_N","Division_W","NewLeague_N"]]],axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [35]:
knn_model = KNeighborsRegressor().fit(x_train,y_train)

In [36]:
knn_model

KNeighborsRegressor()

In [37]:
knn_model.n_neighbors

5

# Tahmin

In [38]:
y_pred_train = knn_model.predict(x_train)

In [39]:
y_pred_test = knn_model.predict(x_test)

In [40]:
np.sqrt(mean_squared_error(y_test,y_pred_test))

426.6570764525201

In [41]:
RMSE = []

for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors= k).fit(x_train,y_train)
    y_pred = knn_model.predict(x_train)
    rmse = np.sqrt(mean_squared_error(y_train,y_pred))
    RMSE.append(rmse)
    print("k = ", k, "için RMSE değeri: ", rmse)

k =  1 için RMSE değeri:  0.0
k =  2 için RMSE değeri:  179.52761335480352
k =  3 için RMSE değeri:  205.20157172291863
k =  4 için RMSE değeri:  220.5139794876305
k =  5 için RMSE değeri:  239.6467132541376
k =  6 için RMSE değeri:  243.5904190007242
k =  7 için RMSE değeri:  258.1478781634636
k =  8 için RMSE değeri:  266.05374203349805
k =  9 için RMSE değeri:  269.73782093553376
k =  10 için RMSE değeri:  271.2798300436963


# Model Tuning 

In [42]:
knn_params = {"n_neighbors": np.arange(1,30,1)}

In [43]:
knn = KNeighborsRegressor()

In [44]:
knn_cv_model = GridSearchCV(knn,knn_params,cv=10)

In [45]:
knn_cv_model.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

In [46]:
knn_cv_model.best_params_["n_neighbors"]

8

In [49]:
RMSE = []
RMSE_CV = []

for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors= k).fit(x_train,y_train)
    y_pred = knn_model.predict(x_train)
    rmse = np.sqrt(mean_squared_error(y_train,y_pred))
    rmse_cv = np.sqrt(-1*cross_val_score(knn_model,x_train,y_train,cv=10,scoring="neg_mean_squared_error").mean())
    RMSE.append(rmse)
    RMSE_CV.append(rmse_cv)
    print("k = ", k, "için RMSE değeri: ", rmse,"RMSE_CV değeri: ",RMSE_CV)

k =  1 için RMSE değeri:  0.0 RMSE_CV değeri:  [325.3947514706382]
k =  2 için RMSE değeri:  179.52761335480352 RMSE_CV değeri:  [325.3947514706382, 293.24000183333817]
k =  3 için RMSE değeri:  205.20157172291863 RMSE_CV değeri:  [325.3947514706382, 293.24000183333817, 283.7486667487823]
k =  4 için RMSE değeri:  220.5139794876305 RMSE_CV değeri:  [325.3947514706382, 293.24000183333817, 283.7486667487823, 286.3240222024089]
k =  5 için RMSE değeri:  239.6467132541376 RMSE_CV değeri:  [325.3947514706382, 293.24000183333817, 283.7486667487823, 286.3240222024089, 290.0705466132226]
k =  6 için RMSE değeri:  243.5904190007242 RMSE_CV değeri:  [325.3947514706382, 293.24000183333817, 283.7486667487823, 286.3240222024089, 290.0705466132226, 298.1263115575851]
k =  7 için RMSE değeri:  258.1478781634636 RMSE_CV değeri:  [325.3947514706382, 293.24000183333817, 283.7486667487823, 286.3240222024089, 290.0705466132226, 298.1263115575851, 294.77070479194987]
k =  8 için RMSE değeri:  266.053742033

In [50]:
knn_tuned = KNeighborsRegressor(knn_cv_model.best_params_["n_neighbors"])

In [51]:
knn_tuned.fit(x_train,y_train)

KNeighborsRegressor(n_neighbors=8)

In [52]:
np.sqrt(mean_squared_error(y_test,knn_tuned.predict(x_test)))

413.7094731463598